In [135]:
import os
import pymysql
conn = pymysql.connect(
    host="211.51.163.232",
    port=19306,
    user="lgup2",
    passwd="lgup2P@ssw0rd",
    database="lgup2",
    local_infile=True,
    cursorclass=pymysql.cursors.DictCursor
)

try: 
    with conn.cursor() as cur:
        # 2) 세션 문자셋 재확인(전송 구간 깨짐 방지)
        cur.execute("SET NAMES utf8mb4;")
        sql = """
        SELECT      *
        FROM        restaurant
        """
    
        cur.execute(sql)
        # 4) 결과 가져오기
        rows = cur.fetchall() # list[dict] / [{id:1}, {location_name:남산타워}]
finally:
    conn.close()

In [136]:
import pandas as pd
restaurant = pd.DataFrame(rows)

In [137]:
restaurant.columns

Index(['restaurant_id', 'restaurant_name', 'place', 'image_path',
       'description', 'Latitude', 'Longitude', 'price', 'near_subway',
       'signature_menu', 'operatingday', 'website', 'type', 'notice',
       'operatinghour', 'tel', 'holidays'],
      dtype='object')

In [ ]:
restaurant['image_path']

0                      images/통인시장_1.jpg
1                       images/삼청각_2.png
2                images/카몽(KAMONG)_3.jpg
3           images/놀부유황오리진흙구이(잠실점)_4.jpg
4                    images/클럽_NB2_5.png
                      ...               
1173            images/아벨롭_홍대본점_1200.png
1174                images/강변서재_1201.png
1175    images/하우스_오브_신세계_디저트살롱_1202.png
1176        images/켄싱턴_호텔_브로드웨이_1203.jpg
1177               images/고봉삼계탕_1204.png
Name: image_path, Length: 1178, dtype: object

In [ ]:
restaurant.to_csv("음식전처리전.csv",encoding = 'utf-8-sig')

In [ ]:
restaurant['description']

0       SBS [런닝맨], KBS [1박2일]의 촬영지이자, 엽전 도시락과 기름 떡볶이로 ...
1       삼청각은 전통예술복합문화공간. 한국의 유명한 대목장 박광석과 대목수 조승원이 참가해...
2       엑소의 멤버 ‘카이가 꿈을 꾸다’ 라는 뜻을 가진 카페 카몽은 카이의 누나가 운영하...
3       황토진흙으로 만든 토기에 유황오리를 넣고, 450。가마에서 3시간 동안 익힌 것.\...
4       한류스타가 즐겨찾는 강남역 거리 및 홍대 클럽 거리에 위치한 대형 힙합댄스 클럽 Y...
                              ...                        
1173    1세대 건축가 김중업이 설계한 베이커리 카페\n\n아벨롭(Avelop)은 전진(ad...
1174    여의도 국회의사당에 위치한 한강뷰 카페\n\n강변서재는 여의도 국회의사당에 위치한 ...
1175    한국의 전통차와 다과 페어링을 즐길 수 있는 프리미엄 전통차 카페\n\n하우스 오브...
1176    올데이 다이닝 레스토랑\n\n뉴욕의 브로드웨이 감성을 그대로 연출한 올데이 다이닝 ...
1177    상황버섯 삼계탕이 유명한 추천 보양식\n\n국회 의사당 인근에 위치한 고봉삼계탕은 ...
Name: description, Length: 1178, dtype: object

In [ ]:
keywords = ["연예인", "셀럽", "촬영", "단골", "유명인", "스타","엑소","틴탑","예능",'TV','인기','드라마',
            '전지현','슈퍼주니어','BTS','국가대표','멤버','배우','영화','외국인']
restaurant["is_celeb"] = restaurant["description"].apply(
    lambda x: any(k in x for k in keywords)
)

# True인 값의 개수 세기
num_celeb = restaurant["is_celeb"].sum()
print(num_celeb)

353


In [ ]:
# True인 행만 선택
celeb_restaurants = restaurant[restaurant["is_celeb"] == True]

# 결과 확인
print(celeb_restaurants)


      restaurant_id restaurant_name  \
0                 1            통인시장   
2                 3      카몽(KAMONG)   
4                 5          클럽 NB2   
8                 9         BHC 사당점   
9                10  씨제이더마켓 제일제당센터점   
...             ...             ...   
1160           1187        태양의토마토라멘   
1164           1191            대포찜닭   
1166           1193          공릉닭한마리   
1169           1196             외양간   
1171           1198             목석정   

                                             place  \
0                         03036  서울 종로구 자하문로15길 18   
2                   06647  서울 서초구 서초대로42길 17 (서초동)   
4                            04041  서울 마포구 와우산로 72   
8                    07015  서울 동작구 동작대로7길 16 (사당동)   
9      04560  서울 중구 동호로 330 (쌍림동, 씨제이제일제당센터)  지하1층   
...                                            ...   
1160                06123  서울특별시 강남구 강남대로106길 7 2층   
1164  03788  서울 서대문구 연세로 27-1 (창천동, 대성빌딩)  대성빌딩 3층   
1166           03778  서울 서대문구 연세로2길 77 (대현동, 

In [ ]:
restaurant["is_celeb"]

0        True
1       False
2        True
3       False
4        True
        ...  
1173    False
1174    False
1175    False
1176    False
1177    False
Name: is_celeb, Length: 1178, dtype: bool

In [ ]:
restaurant.iloc[4]

restaurant_id                                                      5
restaurant_name                                               클럽 NB2
place                                          04041  서울 마포구 와우산로 72
image_path                                       images/클럽_NB2_5.png
description        한류스타가 즐겨찾는 강남역 거리 및 홍대 클럽 거리에 위치한 대형 힙합댄스 클럽 Y...
Latitude                                                   37.550910
Longitude                                                 126.923345
price                                                               
near_subway                                  지하철 6호선 상수역 2번 출구, 341m
signature_menu                                                      
operatingday                                                 월요일~일요일
website                    https://www.instagram.com/clubnb_official
type                                                           술집, 바
notice                                                              
operatinghour               일요일~목요

In [ ]:
import folium

map_center = [celeb_restaurants["Latitude"].mean(), celeb_restaurants["Longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# 마커 추가
for idx, row in celeb_restaurants.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=f"""
        <b>{row['restaurant_name']}</b><br>
        📍 위치: {row['place']}<br>
        🍽 대표 메뉴: {row.get('signature_menu','정보 없음')}<br>
        💰 가격대: {row.get('price','정보 없음')}
        """,
        tooltip=row["restaurant_name"]
    ).add_to(m)

# 지도 저장
m.save("restaurants_map.html")
m

In [ ]:
celeb_restaurants.columns

Index(['restaurant_id', 'restaurant_name', 'place', 'image_path',
       'description', 'Latitude', 'Longitude', 'price', 'near_subway',
       'signature_menu', 'operatingday', 'website', 'type', 'notice',
       'operatinghour', 'tel', 'holidays', 'is_celeb'],
      dtype='object')

In [ ]:
celeb_restaurants.drop(columns =['operatinghour'], inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22800\3967530227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  celeb_restaurants.drop(columns =['operatinghour'], inplace = True)


In [ ]:
celeb_restaurants.drop(columns =['restaurant_id'], inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22800\36003656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  celeb_restaurants.drop(columns =['restaurant_id'], inplace = True)


In [ ]:
import pandas as pd

# NaN + 공백("") 개수 세기
def count_missing_or_blank(df):
    missing_counts = {}
    for col in df.columns:
        # NaN 개수 + 공백 문자열 개수 합산
        missing = df[col].isna().sum() + (df[col] == "").sum()
        missing_counts[col] = missing
    return pd.Series(missing_counts)

# 실행
missing_per_column = count_missing_or_blank(celeb_restaurants)
print(missing_per_column)


restaurant_id       0
restaurant_name     0
place               0
image_path          0
description         0
Latitude            0
Longitude           0
near_subway         1
type               27
dtype: int64


In [ ]:
celeb_restaurants

,restaurant_name,place,image_path,description,Latitude,Longitude,near_subway,type
0,통인시장,03036 서울 종로구 자하문로15길 18,images/통인시장_1.jpg,"SBS [런닝맨], KBS [1박2일]의 촬영지이자, 엽전 도시락과 기름 떡볶이로 ...",37.580755,126.969996,"지하철 3호선 경복궁역 2번 출구, 515m",음식점
2,카몽(KAMONG),06647 서울 서초구 서초대로42길 17 (서초동),images/카몽(KAMONG)_3.jpg,엑소의 멤버 ‘카이가 꿈을 꾸다’ 라는 뜻을 가진 카페 카몽은 카이의 누나가 운영하...,37.491208,127.008940,2호선 서초역 1번 출구에서 약 120m (도보 1분),"카페, 디저트"
4,클럽 NB2,04041 서울 마포구 와우산로 72,images/클럽_NB2_5.png,한류스타가 즐겨찾는 강남역 거리 및 홍대 클럽 거리에 위치한 대형 힙합댄스 클럽 Y...,37.550910,126.923345,"지하철 6호선 상수역 2번 출구, 341m","술집, 바"
8,BHC 사당점,07015 서울 동작구 동작대로7길 16 (사당동),images/BHC_사당점_9.jpg,BHC 사당점은 틴탑 리키의 부모님이 최근 오픈한 치킨 전문점. 지하 1층부터 옥상...,37.479923,126.980702,"지하철 2 / 4호선 사당역 10번 출구, 244m",음식점
9,씨제이더마켓 제일제당센터점,"04560 서울 중구 동호로 330 (쌍림동, 씨제이제일제당센터) 지하1층",images/씨제이더마켓_제일제당센터점_10.jpg,"특별한 미식과 새로운 라이프스타일, 식문화 트렌드를 경험할 수 있는 복합 식문화 공...",37.563974,127.003253,"지하철 2/4/5호선 동대문역사문화공원역 6번 출구, 118m",음식점
...,...,...,...,...,...,...,...,...
1160,태양의토마토라멘,06123 서울특별시 강남구 강남대로106길 7 2층,images/태양의토마토라멘_1187.jpg,"일본에서 넘어온 트렌디한 라멘, 태양의 토마토라멘\n\n태양의토마토라멘은 일본식 라...",37.502977,127.026224,지하철 9호선/신분당선 신논현역 6번출구 137m,
1164,대포찜닭,"03788 서울 서대문구 연세로 27-1 (창천동, 대성빌딩) 대성빌딩 3층",images/대포찜닭_1191.jpg,닭 인형으로 주문하고 닭 캐릭터로 인테리어를 하는 등 색다른 경험을 제공하여 젊은 ...,37.558055,126.936714,"지하철 2호선 신촌역 2번 출구, 278m",음식점
1166,공릉닭한마리,"03778 서울 서대문구 연세로2길 77 (대현동, 부소빌딩)",images/공릉닭한마리_1193.jpg,촉촉하고 야들한 닭고기를 양념장에 찍어 먹고 깊은 국물까지 떠먹으면 닭한마리의 맛을...,37.557129,126.941073,"지하철 경의중앙선 신촌역 1번 출구, 417m",음식점
1169,외양간,"07014 서울 동작구 동작대로11길 16 (사당동, HS하우스)",images/외양간_1196.jpg,대기줄이 언제나 길 만큼 인기 있는 고기요리 전문점입니다. 특수부위 모듬과 키조개 ...,37.481258,126.980728,"지하철 2, 4호선 사당역 10번 출구, 408m",


In [ ]:
import pandas as pd

# 예시: 줄바꿈 제거 + 양쪽 공백 제거
celeb_restaurants["description_clean"] = celeb_restaurants["description"].apply(
    lambda x: str(x).replace("\n", " ").replace("\r", " ").strip()
)

# 연속 공백 하나로 통일
celeb_restaurants["description_clean"] = celeb_restaurants["description_clean"].str.replace(r"\s+", " ", regex=True)


C:\Users\Admin\AppData\Local\Temp\ipykernel_22800\3085295984.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  celeb_restaurants["description_clean"] = celeb_restaurants["description"].apply(
C:\Users\Admin\AppData\Local\Temp\ipykernel_22800\3085295984.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  celeb_restaurants["description_clean"] = celeb_restaurants["description_clean"].str.replace(r"\s+", " ", regex=True)


In [ ]:
celeb_restaurants

,restaurant_name,place,image_path,description,Latitude,Longitude,near_subway,type,description_clean
0,통인시장,03036 서울 종로구 자하문로15길 18,images/통인시장_1.jpg,"SBS [런닝맨], KBS [1박2일]의 촬영지이자, 엽전 도시락과 기름 떡볶이로 ...",37.580755,126.969996,"지하철 3호선 경복궁역 2번 출구, 515m",음식점,"SBS [런닝맨], KBS [1박2일]의 촬영지이자, 엽전 도시락과 기름 떡볶이로 ..."
2,카몽(KAMONG),06647 서울 서초구 서초대로42길 17 (서초동),images/카몽(KAMONG)_3.jpg,엑소의 멤버 ‘카이가 꿈을 꾸다’ 라는 뜻을 가진 카페 카몽은 카이의 누나가 운영하...,37.491208,127.008940,2호선 서초역 1번 출구에서 약 120m (도보 1분),"카페, 디저트",엑소의 멤버 ‘카이가 꿈을 꾸다’ 라는 뜻을 가진 카페 카몽은 카이의 누나가 운영하...
4,클럽 NB2,04041 서울 마포구 와우산로 72,images/클럽_NB2_5.png,한류스타가 즐겨찾는 강남역 거리 및 홍대 클럽 거리에 위치한 대형 힙합댄스 클럽 Y...,37.550910,126.923345,"지하철 6호선 상수역 2번 출구, 341m","술집, 바",한류스타가 즐겨찾는 강남역 거리 및 홍대 클럽 거리에 위치한 대형 힙합댄스 클럽 Y...
8,BHC 사당점,07015 서울 동작구 동작대로7길 16 (사당동),images/BHC_사당점_9.jpg,BHC 사당점은 틴탑 리키의 부모님이 최근 오픈한 치킨 전문점. 지하 1층부터 옥상...,37.479923,126.980702,"지하철 2 / 4호선 사당역 10번 출구, 244m",음식점,BHC 사당점은 틴탑 리키의 부모님이 최근 오픈한 치킨 전문점. 지하 1층부터 옥상...
9,씨제이더마켓 제일제당센터점,"04560 서울 중구 동호로 330 (쌍림동, 씨제이제일제당센터) 지하1층",images/씨제이더마켓_제일제당센터점_10.jpg,"특별한 미식과 새로운 라이프스타일, 식문화 트렌드를 경험할 수 있는 복합 식문화 공...",37.563974,127.003253,"지하철 2/4/5호선 동대문역사문화공원역 6번 출구, 118m",음식점,"특별한 미식과 새로운 라이프스타일, 식문화 트렌드를 경험할 수 있는 복합 식문화 공..."
...,...,...,...,...,...,...,...,...,...
1160,태양의토마토라멘,06123 서울특별시 강남구 강남대로106길 7 2층,images/태양의토마토라멘_1187.jpg,"일본에서 넘어온 트렌디한 라멘, 태양의 토마토라멘\n\n태양의토마토라멘은 일본식 라...",37.502977,127.026224,지하철 9호선/신분당선 신논현역 6번출구 137m,,"일본에서 넘어온 트렌디한 라멘, 태양의 토마토라멘 태양의토마토라멘은 일본식 라멘에 ..."
1164,대포찜닭,"03788 서울 서대문구 연세로 27-1 (창천동, 대성빌딩) 대성빌딩 3층",images/대포찜닭_1191.jpg,닭 인형으로 주문하고 닭 캐릭터로 인테리어를 하는 등 색다른 경험을 제공하여 젊은 ...,37.558055,126.936714,"지하철 2호선 신촌역 2번 출구, 278m",음식점,닭 인형으로 주문하고 닭 캐릭터로 인테리어를 하는 등 색다른 경험을 제공하여 젊은 ...
1166,공릉닭한마리,"03778 서울 서대문구 연세로2길 77 (대현동, 부소빌딩)",images/공릉닭한마리_1193.jpg,촉촉하고 야들한 닭고기를 양념장에 찍어 먹고 깊은 국물까지 떠먹으면 닭한마리의 맛을...,37.557129,126.941073,"지하철 경의중앙선 신촌역 1번 출구, 417m",음식점,촉촉하고 야들한 닭고기를 양념장에 찍어 먹고 깊은 국물까지 떠먹으면 닭한마리의 맛을...
1169,외양간,"07014 서울 동작구 동작대로11길 16 (사당동, HS하우스)",images/외양간_1196.jpg,대기줄이 언제나 길 만큼 인기 있는 고기요리 전문점입니다. 특수부위 모듬과 키조개 ...,37.481258,126.980728,"지하철 2, 4호선 사당역 10번 출구, 408m",,대기줄이 언제나 길 만큼 인기 있는 고기요리 전문점입니다. 특수부위 모듬과 키조개 ...


In [ ]:
celeb_restaurants.to_csv("ver2.csv",encoding = 'utf-8-sig')

In [ ]:
food = pd.read_csv("ver2.csv")

In [ ]:
food.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
food.to_csv("food_trans_pre.csv")

In [ ]:
food = pd.read_csv("food_trans_pre.csv")

In [ ]:
food.rename(columns={'Unnamed: 0': 'restaurant_id'}, inplace=True)

In [ ]:
food.to_csv("food_trans_pre.csv")

In [ ]:
food.columns

Index(['restaurant_id', 'restaurant_name', 'place', 'image_path', 'Latitude',
       'Longitude', 'near_subway', 'type', 'description_clean'],
      dtype='object')

In [ ]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from time import sleep
import json

load_dotenv()

INPUT_FILE = 'food_trans_pre.csv'
OUTPUT_FILE = 'translated_food_papago.csv'
CACHE_FILE = 'papago_cache.json'  # ✅ 캐시 파일

NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID") or "."
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET") or "."

API_URL = "https://papago.apigw.ntruss.com/nmt/v1/translation"

DATA_COLS_TO_TRANSLATE = [
    'restaurant_name', 'place',
     'near_subway', 'type', 'description_clean'
]

# ✅ 캐시 로드
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, 'r', encoding='utf-8') as f:
        cache = json.load(f)
else:
    cache = {}

def save_cache():
    """✅ 캐시 저장 함수"""
    with open(CACHE_FILE, 'w', encoding='utf-8') as f:
        json.dump(cache, f, ensure_ascii=False, indent=2)

def translate_papago(text: str, source_lang='ko', target_lang='en') -> str:
    if pd.isna(text) or not str(text).strip():
        return text

    text = str(text).strip()

    # ✅ 캐시 체크
    if text in cache:
        return cache[text]

    headers = {
        "X-NCP-APIGW-API-KEY-ID": NAVER_CLIENT_ID,
        "X-NCP-APIGW-API-KEY": NAVER_CLIENT_SECRET,
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
    }

    data = {
        'source': source_lang,
        'target': target_lang,
        'text': text
    }

    try:
        response = requests.post(API_URL, headers=headers, data=data, timeout=15)

        if response.status_code == 429:
            print("⚠️ 요청 한도 초과! 1초 대기 후 재시도...")
            sleep(1)
            return translate_papago(text)

        response.raise_for_status()
        result = response.json()

        translated = result['message']['result']['translatedText']

        # ✅ 캐시에 저장
        cache[text] = translated
        save_cache()

        return translated

    except Exception as e:
        print(f"❌ 번역 오류: {text} → {e}")
        return text


def translate_csv_papago(input_path, output_path, col_list):
    try:
        df = pd.read_csv(input_path)
    except:
        print("❌ 입력 CSV 파일을 찾을 수 없습니다.")
        return

    print("\n✅ [1] 헤더 번역 시작")
    header_map = {}
    for c in df.columns:
        t = translate_papago(c)
        header_map[c] = t
        print(f"Header: {c} → {t}")
        sleep(0.1)
    df.rename(columns=header_map, inplace=True)

    print("\n✅ [2] 내용 번역 시작")
    for col in col_list:
        if col in df.columns:
            new_col = f"{col}_en"
            print(f"Column: {col} → {new_col}")
            df[new_col] = df[col].apply(translate_papago)
            save_cache()  # ✅ 중간 결과 저장
        else:
            print(f"⚠️ 해당 컬럼 없음: {col}")

    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    save_cache()

    print(f"\n🎉 번역 완료! 저장 위치: {output_path}")
    print(f"📌 캐시 저장: {CACHE_FILE}")


translate_csv_papago(INPUT_FILE, OUTPUT_FILE, DATA_COLS_TO_TRANSLATE)


✅ [1] 헤더 번역 시작
Header: Unnamed: 0 → Unnamed: 0
Header: restaurant_id → restaurant_id
Header: restaurant_name → restaurant_name
Header: place → place
Header: image_path → image_path
Header: Latitude → Latitude
Header: Longitude → Longitude
Header: near_subway → near_subway
Header: type → type
Header: description_clean → description_clean

✅ [2] 내용 번역 시작
Column: restaurant_name → restaurant_name_en
Column: place → place_en
Column: near_subway → near_subway_en
Column: type → type_en
Column: description_clean → description_clean_en

🎉 번역 완료! 저장 위치: translated_food_papago.csv
📌 캐시 저장: papago_cache.json


In [144]:
food = pd.read_csv("translated_food_papago.csv")

In [145]:
food.columns

Index(['restaurant_id', 'restaurant_name', 'place', 'image_path', 'Latitude',
       'Longitude', 'near_subway', 'type', 'description_clean',
       'restaurant_name_en', 'place_en', 'near_subway_en', 'type_en',
       'description_clean_en'],
      dtype='object')

In [ ]:
import pymysql

conn = pymysql.connect(
    host=".",
    port=.,
    user="lgup2",
    passwd="lgup2P@ssw0rd",
    database="lgup2",
    local_infile=True,
    cursorclass=pymysql.cursors.DictCursor
)

create_table_sql = """
CREATE TABLE IF NOT EXISTS celeb_restaurants (
    restaurant_id INT PRIMARY KEY,
    restaurant_name VARCHAR(255),
    place VARCHAR(255),
    image_path TEXT,
    Latitude DOUBLE,
    Longitude DOUBLE,
    near_subway VARCHAR(255),
    type VARCHAR(255),
    description_clean TEXT,
    restaurant_name_en VARCHAR(255),
    place_en VARCHAR(255),
    near_subway_en VARCHAR(255),
    type_en VARCHAR(255),
    description_clean_en TEXT
);
"""

with conn.cursor() as cur:
    cur.execute(create_table_sql)
    conn.commit()

conn.close()
print("✅ celeb_restaurants 테이블 생성 완료!")


✅ celeb_restaurants 테이블 생성 완료!


In [148]:
import pymysql

# 🔹 NaN 처리
for col in food.columns:
    if food[col].dtype == 'float64' or food[col].dtype == 'int64':
        food[col] = food[col].where(food[col].notna(), None)
    else:
        food[col] = food[col].fillna('')

conn = pymysql.connect(
    host="211.51.163.232",
    port=19306,
    user="lgup2",
    passwd="lgup2P@ssw0rd",
    database="lgup2",
    local_infile=True,
    cursorclass=pymysql.cursors.DictCursor
)

insert_sql = """
INSERT INTO celeb_restaurants (
    restaurant_id, restaurant_name, place, image_path,
    Latitude, Longitude, near_subway, type, description_clean,
    restaurant_name_en, place_en, near_subway_en, type_en,
    description_clean_en
)
VALUES (
    %(restaurant_id)s, %(restaurant_name)s, %(place)s, %(image_path)s,
    %(Latitude)s, %(Longitude)s, %(near_subway)s, %(type)s, %(description_clean)s,
    %(restaurant_name_en)s, %(place_en)s, %(near_subway_en)s, %(type_en)s,
    %(description_clean_en)s
)
"""

try:
    with conn.cursor() as cur:
        cur.executemany(insert_sql, food.to_dict('records'))
        conn.commit()
        print("✅ 데이터 Insert 완료!")

except Exception as e:
    print("🚨 오류 발생:", e)

finally:
    conn.close()


✅ 데이터 Insert 완료!
